In [1]:
import subprocess
import json

%run py_feature_extraction.ipynb
from py_constants import CODE_INPUT, XML_OUTPUT

## COMMON Methods

In [2]:
def parse_xml(xml_file):
    """Parse the XML file and return the root element."""
    tree = ET.parse(xml_file)
    return tree.getroot()

def generate_ast(source_code_file, output_file):
    subprocess.run(['srcml', source_code_file, '-o', output_file])

In [3]:
def get_quality_metrics(root):
    # Construct the JSON object
    quality_metrics = {
        'NF': count_function_declarations_excluding_main(root),
        'NP': get_parameter_count_in_functions(root),
        'NPt-s': count_simple_pointers(root),
        'NAdd-s': count_simple_addresses(root),
        'NPt-ds': count_array_pointers(root),
        'NStr': count_structs(root),
        'NStrM': count_struct_members(root),
        'NStrT': count_struct_typedefs(root),
        'NStrI': count_struct_instances(root),
        'NStrC': count_struct_calls(root),
        'NFRec': count_recursive_functions(root),
        'NCRec': count_recursive_function_calls(root),
        'NIFPar': count_parameters_in_conditional(root),
        'NRRec': count_recursive_returns(root),
        'NRNRec': count_non_recursive_returns(root),
        'NMalloc': count_malloc_functions(root),
        'NSizeof': count_sizeof(root),
        'NFree': count_free_functions(root)
    }
    print(quality_metrics)
    # return json.dumps(quality_metrics, indent=4)
    quality_metrics_filtered = {k: v for k, v in quality_metrics.items() if v >= 1}

    return quality_metrics_filtered

In [4]:
FUNCTION_MAP = {
    'NF': count_function_declarations_excluding_main,
    'NP': get_parameter_count_in_functions,
    'NPt-s': count_simple_pointers,
    'NAdd-s': count_simple_addresses,
    'NPt-ds': count_array_pointers,
    'NStr': count_structs,
    'NStrM': count_struct_members,
    'NStrT': count_struct_typedefs,
    'NStrI': count_struct_instances,
    'NStrC': count_struct_calls,
    'NFRec': count_recursive_functions,
    'NCRec': count_recursive_function_calls,
    'NIFPar': count_parameters_in_conditional,
    'NRRec': count_recursive_returns,
    'NRNRec': count_non_recursive_returns,
    'NMalloc': count_malloc_functions,
    'NSizeof': count_sizeof,
    'NFree': count_free_functions
}

In [5]:
def extract_features(root,json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    features = {}
    for key, func in FUNCTION_MAP.items():
        if key in data:
            features[key] = func(root)
    
    return features

## Main Function

In [6]:
if __name__ == "__main__":
    # Specify the source code file and output file for the AST
    #source_code_file = 'test.cpp'
    #xml_output_file = 'ast.xml'

    # Generate AST
    #generate_ast(source_code_file, xml_output_file)

    # Example usage
    xml_file = CODE_INPUT+"/ast.xml"
    #xml_file = CODE_INPUT+"/pointers.xml"
    #xml_file = CODE_INPUT+"/arrPointer.xml"
    #xml_file = CODE_INPUT+"/structExample.xml"
    #xml_file = CODE_INPUT+"/typedefExample.xml"
    #xml_file = CODE_INPUT+"/recursionExample.xml"
    #xml_file = CODE_INPUT+"/mallocExample.xml"
    #xml_file = CODE_INPUT+"/sizeofExample.xml"

    root =  parse_xml(xml_file)
    #%run py_feature_extraction.ipynb
    
    json_metrics = get_quality_metrics(root)
    json_file = "quality_metrics.json"
    with open(json_file, "w") as json_file:
        json.dump(json_metrics, json_file, indent=4)

    print(f"Quality metrics saved to {json_file}")
    #print("printing json_file:",json_file.name)
    extracted_features = extract_features(root, json_file.name)

    print("Extracted features:")
    print(extracted_features)

printing struct_declarations:  {}
printing struct_inst_name:  {}
printing function_bodies: {'isPrime': ['sqrt'], 'factorial': ['factorial'], 'fibonacci': [], 'main': ['isPrime', 'factorial', 'fibonacci']}
printing recursive_functions:  ['factorial']
printing recursive_functions:  ['factorial']
printing recursive_functions:  ['factorial']
printing recursive_functions:  ['factorial']
{'NF': 3, 'NP': 3, 'NPt-s': 0, 'NAdd-s': 0, 'NPt-ds': 0, 'NStr': 0, 'NStrM': 0, 'NStrT': 0, 'NStrI': 0, 'NStrC': 0, 'NFRec': 1, 'NCRec': 1, 'NIFPar': 1, 'NRRec': 1, 'NRNRec': 1, 'NMalloc': 0, 'NSizeof': 0, 'NFree': 0}
Quality metrics saved to <_io.TextIOWrapper name='quality_metrics.json' mode='w' encoding='cp1252'>
printing function_bodies: {'isPrime': ['sqrt'], 'factorial': ['factorial'], 'fibonacci': [], 'main': ['isPrime', 'factorial', 'fibonacci']}
printing recursive_functions:  ['factorial']
printing recursive_functions:  ['factorial']
printing recursive_functions:  ['factorial']
printing recursive_fun

In [7]:
def calculate_individual_scores(teacher_solution, student_solution):
    individual_scores = {}
    
    for feature, teacher_count in teacher_solution.items():
        if feature in student_solution:
            student_count = student_solution[feature]
            # Calculate the score based on the difference between counts
            feature_score = max(0, min(1, 1 - abs(teacher_count - student_count) / teacher_count))
            # Store the individual score for the feature
            individual_scores[feature] = f"{student_count}/{teacher_count}"
        else:
            individual_scores[feature] = f"0/{teacher_count}"
    
    return individual_scores

# Example usage:
teacher_solution = {'NF': 5, 'NP': 10, 'NPt-s': 3, 'NAdd-s': 2, 'NPt-ds': 1}
student_solution = {'NF': 4, 'NP': 9, 'NPt-s': 3, 'NAdd-s': 1, 'NPt-ds': 1}

individual_scores = calculate_individual_scores(teacher_solution, student_solution)
print("Individual scores:", individual_scores)


Individual scores: {'NF': '4/5', 'NP': '9/10', 'NPt-s': '3/3', 'NAdd-s': '1/2', 'NPt-ds': '1/1'}


In [8]:
def calculate_individual_scores(teacher_solution, student_solution):
    individual_scores = {}
    total_score = 0
    
    for feature, teacher_count in teacher_solution.items():
        if feature in student_solution:
            student_count = student_solution[feature]
            # Calculate the score based on the difference between counts
            feature_score = max(0, min(1, 1 - abs(teacher_count - student_count) / teacher_count))
            # Add the feature score to the total score
            total_score += feature_score
            # Store the individual score for the feature
            individual_scores[feature] = {
                "student_count": student_count,
                "teacher_count": teacher_count,
                "score" : f"{student_count}/{teacher_count}",
                "feature_score": feature_score
            }
        else:
            # If the feature is not present in the student's solution
            individual_scores[feature] = {
                "student_count": 0,
                "teacher_count": teacher_count,
                "feature_score": 0
            }
    
    # Calculate the final score percentage
    num_features = len(teacher_solution)
    final_score_percentage = total_score / num_features * 100 if num_features > 0 else 0
    
    return individual_scores, final_score_percentage

# Example usage:
teacher_solution = {'NF': 5, 'NP': 10, 'NPt-s': 3, 'NAdd-s': 2, 'NPt-ds': 1}
student_solution = {'NF': 4, 'NP': 9, 'NPt-s': 3, 'NAdd-s': 1, 'NPt-ds': 1}

individual_scores, final_score_percentage = calculate_individual_scores(teacher_solution, student_solution)
print("Individual scores:", individual_scores)
print("Final score percentage:", final_score_percentage)


Individual scores: {'NF': {'student_count': 4, 'teacher_count': 5, 'score': '4/5', 'feature_score': 0.8}, 'NP': {'student_count': 9, 'teacher_count': 10, 'score': '9/10', 'feature_score': 0.9}, 'NPt-s': {'student_count': 3, 'teacher_count': 3, 'score': '3/3', 'feature_score': 1}, 'NAdd-s': {'student_count': 1, 'teacher_count': 2, 'score': '1/2', 'feature_score': 0.5}, 'NPt-ds': {'student_count': 1, 'teacher_count': 1, 'score': '1/1', 'feature_score': 1}}
Final score percentage: 84.00000000000001


In [9]:
def calculate_individual_scores(teacher_solution, student_solution):
    individual_scores = {}
    total_score = 0
    
    for feature, teacher_count in teacher_solution.items():
        if feature in student_solution:
            student_count = student_solution[feature]
            # Calculate the score based on the difference between counts
            feature_score = max(0, min(1, 1 - abs(teacher_count - student_count) / teacher_count))
            # Penalize if the student's count exceeds the teacher's count
            if student_count > teacher_count:
                feature_score *= 0.5  # Example penalty factor (adjust as needed)
            # Add the feature score to the total score
            total_score += feature_score
            # Store the individual score for the feature
            individual_scores[feature] = {
                "student_count": student_count,
                "teacher_count": teacher_count,
                "score": f"{student_count}/{teacher_count}",
                "feature_score": feature_score
            }
        else:
            # If the feature is not present in the student's solution
            individual_scores[feature] = {
                "student_count": 0,
                "teacher_count": teacher_count,
                "score": f"0/{teacher_count}",
                "feature_score": 0
            }
    
    # Calculate the final score percentage
    num_features = len(teacher_solution)
    final_score_percentage = total_score / num_features * 100 if num_features > 0 else 0
    print(total_score,num_features)
    return individual_scores, final_score_percentage

# Example usage:
teacher_solution = {'NF': 5, 'NP': 10, 'NPt-s': 3, 'NAdd-s': 2, 'NPt-ds': 1}
student_solution = {'NF': 4, 'NP': 12, 'NPt-s': 2, 'NAdd-s': 3, 'NPt-ds': 1}

individual_scores, final_score_percentage = calculate_individual_scores(teacher_solution, student_solution)
print("Individual scores:", individual_scores)
print("Final score percentage:", final_score_percentage)


3.116666666666667 5
Individual scores: {'NF': {'student_count': 4, 'teacher_count': 5, 'score': '4/5', 'feature_score': 0.8}, 'NP': {'student_count': 12, 'teacher_count': 10, 'score': '12/10', 'feature_score': 0.4}, 'NPt-s': {'student_count': 2, 'teacher_count': 3, 'score': '2/3', 'feature_score': 0.6666666666666667}, 'NAdd-s': {'student_count': 3, 'teacher_count': 2, 'score': '3/2', 'feature_score': 0.25}, 'NPt-ds': {'student_count': 1, 'teacher_count': 1, 'score': '1/1', 'feature_score': 1}}
Final score percentage: 62.33333333333334
